# Setup

In [1]:
%load_ext autoreload
%autoreload 2

# Monkey with the path, so that we can import like rational humans
try:
    import deepseg as ds
except:
    import sys, os
    sys.path.insert(0, os.path.abspath('./../../trace/'))

import tensorflow as tf
import numpy as np
import pandas

import trace.common as com
import trace.sampling as samp
import trace.dataset as dat
import trace.train as train
import trace.models as mod
import trace.train.hooks as hooks
import trace.evaluation as eva

from common import showarray, get_metrics, print_and_save_metrics, load_classifier, run_full_training

In [2]:
dataset_folder = './../../../datasets/'
# Dataframe with results
df_snemi = pandas.DataFrame(index=['VD2D-3D', 'UVR-Net'],
                            columns=['Pixel Error', 'Rand - Full', 'Rand - Merge', 'Rand - Split'])

df_cremi_a = pandas.DataFrame(index=['VD2D-3D', 'UVR-Net'], 
                              columns=['Pixel Error', 'Rand - Full', 'Rand - Merge', 'Rand - Split'])

df_cremi_b = pandas.DataFrame(index=['VD2D-3D', 'UVR-Net'], 
                              columns=['Pixel Error', 'Rand - Full', 'Rand - Merge', 'Rand - Split'])

df_cremi_c = pandas.DataFrame(index=['VD2D-3D', 'UVR-Net'], 
                              columns=['Pixel Error', 'Rand - Full', 'Rand - Merge', 'Rand - Split'])

# Model Training

## SNEMI3D

### VD2D-3D

In [7]:
tf.reset_default_graph()

In [8]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/snemi3d/',
    dataset_constructor=dat.SNEMI3DDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.VD2D_3D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='vd2d_3d'
)


In [9]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//snemi3d/results/vd2d_3d/run-1/model.ckpt
Model restored.


In [10]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D-3D', df_snemi, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=0, x=640
z=0, y=0, x=800
z=0, y=0, x=960
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=160, x=640
z=0, y=160, x=800
z=0, y=160, x=960
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=320, x=640
z=0, y=320, x=800
z=0, y=320, x=960
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
z=0, y=480, x=640
z=0, y=480, x=800
z=0, y=480, x=960
z=0, y=640, x=0
z=0, y=640, x=160
z=0, y=640, x=320
z=0, y=640, x=480
z=0, y=640, x=640
z=0, y=640, x=800
z=0, y=640, x=960
z=0, y=800, x=0
z=0, y=800, x=160
z=0, y=800, x=320
z=0, y=800, x=480
z=0, y=800, x=640
z=0, y=800, x=800
z=0, y=800, x=960
z=0, y=960, x=0
z=0, y=960, x=160
z=0, y=960, x=320
z=0, y=960, x=480
z=0, y=960, x=640
z=0, y=960, x=800
z=0, y=960, x=960
z=16, y=0, x=0
z=16, y=0, x=160
z=16, y=0, x=320
z=16, y=0, x=480
z=16, y=0, x=640
z=16, y=0, x=800
z=16, y=0, x=960
z=16, y=160, x=0
z=16, y=160,

### U-Net

In [3]:
tf.reset_default_graph()

In [4]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/snemi3d/',
    dataset_constructor=dat.SNEMI3DDataset,
    model_constructor=mod.UNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.UNET_3D_4LAYERS,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='unet-3d-4layers'
)

In [5]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='2')
classifier, dset_sampler = load_classifier(pipeline, run_name='2')

INFO:tensorflow:Restoring parameters from ./../../../datasets//snemi3d/results/unet_3d_4layers/run-2/model.ckpt
Model restored.


In [6]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('UVR-Net', df_snemi, p_error, rand_full, rand_merge, rand_split)

z: 0/24
y: 0/1023
y: 150/1023
y: 300/1023
y: 450/1023
y: 600/1023
y: 750/1023
y: 863/1023
z: 8/24
y: 0/1023
y: 150/1023
y: 300/1023
y: 450/1023
y: 600/1023
y: 750/1023
y: 863/1023
(24, 1023, 1023)
Begin segmentation
Segmentation complete
Performing foreground-restriction
Finding overlap matrix...
Completed in 4.578804 seconds
Calculating Rand F-Score...
Completed in 0.077447 seconds
UVR-Net
Pixel Error: 0.086327
Rand - Full: 0.783288
Rand - Merge: 0.993469
Rand - Split: 0.646511


## CREMI

### CREMI A Unaligned, with VD2D-3D

In [44]:
tf.reset_default_graph()

In [45]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/a/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.VD2D_3D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='vd2d-3d'
)

In [46]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/a/results/vd2d_3d/run-1/model.ckpt
Model restored.


In [47]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D-3D', df_cremi_a, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=0, x=640
z=0, y=0, x=800
z=0, y=0, x=960
z=0, y=0, x=1120
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=160, x=640
z=0, y=160, x=800
z=0, y=160, x=960
z=0, y=160, x=1120
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=320, x=640
z=0, y=320, x=800
z=0, y=320, x=960
z=0, y=320, x=1120
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
z=0, y=480, x=640
z=0, y=480, x=800
z=0, y=480, x=960
z=0, y=480, x=1120
z=0, y=640, x=0
z=0, y=640, x=160
z=0, y=640, x=320
z=0, y=640, x=480
z=0, y=640, x=640
z=0, y=640, x=800
z=0, y=640, x=960
z=0, y=640, x=1120
z=0, y=800, x=0
z=0, y=800, x=160
z=0, y=800, x=320
z=0, y=800, x=480
z=0, y=800, x=640
z=0, y=800, x=800
z=0, y=800, x=960
z=0, y=800, x=1120
z=0, y=960, x=0
z=0, y=960, x=160
z=0, y=960, x=320
z=0, y=960, x=480
z=0, y=960, x=640
z=0, y=960, x=800
z=0, y=960, x=960
z=0, y=960, x=1120
z=0, y=1120, x=

### CREMI A Unaligned, with U-Net 3D

In [23]:
tf.reset_default_graph()

In [24]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/a/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.UNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.UNET_3D_4LAYERS,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='unet-3d-4layers'
)

In [25]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='2')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/a/results/unet_3d_4layers/run-2/model.ckpt
Model restored.


In [26]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('UVR-Net', df_cremi_a, p_error, rand_full, rand_merge, rand_split)

z: 0/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
z: 8/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
(24, 1249, 1249)
Begin segmentation
Segmentation complete
Performing foreground-restriction
Finding overlap matrix...
Completed in 7.531078 seconds
Calculating Rand F-Score...
Completed in 0.029199 seconds
UVR-Net
Pixel Error: 0.034879
Rand - Full: 0.576646
Rand - Merge: 0.449545
Rand - Split: 0.803948


### Cremi B unaligned, VD2D-3D

In [40]:
tf.reset_default_graph()

In [41]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/b/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.VD2D_3D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='vd2d-3d'
)

In [42]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/b/results/vd2d_3d/run-1/model.ckpt
Model restored.


In [43]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D-3D', df_cremi_b, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=0, x=640
z=0, y=0, x=800
z=0, y=0, x=960
z=0, y=0, x=1120
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=160, x=640
z=0, y=160, x=800
z=0, y=160, x=960
z=0, y=160, x=1120
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=320, x=640
z=0, y=320, x=800
z=0, y=320, x=960
z=0, y=320, x=1120
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
z=0, y=480, x=640
z=0, y=480, x=800
z=0, y=480, x=960
z=0, y=480, x=1120
z=0, y=640, x=0
z=0, y=640, x=160
z=0, y=640, x=320
z=0, y=640, x=480
z=0, y=640, x=640
z=0, y=640, x=800
z=0, y=640, x=960
z=0, y=640, x=1120
z=0, y=800, x=0
z=0, y=800, x=160
z=0, y=800, x=320
z=0, y=800, x=480
z=0, y=800, x=640
z=0, y=800, x=800
z=0, y=800, x=960
z=0, y=800, x=1120
z=0, y=960, x=0
z=0, y=960, x=160
z=0, y=960, x=320
z=0, y=960, x=480
z=0, y=960, x=640
z=0, y=960, x=800
z=0, y=960, x=960
z=0, y=960, x=1120
z=0, y=1120, x=

### Cremi B unaligned, U-Net 3D

In [27]:
tf.reset_default_graph()

In [28]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/b/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.UNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.UNET_3D_4LAYERS,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='unet-3d-4layers'
)

In [29]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/b/results/unet_3d_4layers/run-1/model.ckpt
Model restored.


In [30]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('UVR-Net', df_cremi_b, p_error, rand_full, rand_merge, rand_split)

z: 0/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
z: 8/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
(24, 1249, 1249)
Begin segmentation
Segmentation complete
Performing foreground-restriction
Finding overlap matrix...
Completed in 7.672391 seconds
Calculating Rand F-Score...
Completed in 0.030734 seconds
UVR-Net
Pixel Error: 0.037926
Rand - Full: 0.846339
Rand - Merge: 0.933393
Rand - Split: 0.774138


### Cremi C unaligned, VD2D-3D

In [31]:
tf.reset_default_graph()

In [32]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/c/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.VD2D_3D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='vd2d-3d'
)

In [33]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/c/results/vd2d_3d/run-1/model.ckpt
Model restored.


In [34]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D-3D', df_cremi_c, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=0, x=640
z=0, y=0, x=800
z=0, y=0, x=960
z=0, y=0, x=1120
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=160, x=640
z=0, y=160, x=800
z=0, y=160, x=960
z=0, y=160, x=1120
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=320, x=640
z=0, y=320, x=800
z=0, y=320, x=960
z=0, y=320, x=1120
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
z=0, y=480, x=640
z=0, y=480, x=800
z=0, y=480, x=960
z=0, y=480, x=1120
z=0, y=640, x=0
z=0, y=640, x=160
z=0, y=640, x=320
z=0, y=640, x=480
z=0, y=640, x=640
z=0, y=640, x=800
z=0, y=640, x=960
z=0, y=640, x=1120
z=0, y=800, x=0
z=0, y=800, x=160
z=0, y=800, x=320
z=0, y=800, x=480
z=0, y=800, x=640
z=0, y=800, x=800
z=0, y=800, x=960
z=0, y=800, x=1120
z=0, y=960, x=0
z=0, y=960, x=160
z=0, y=960, x=320
z=0, y=960, x=480
z=0, y=960, x=640
z=0, y=960, x=800
z=0, y=960, x=960
z=0, y=960, x=1120
z=0, y=1120, x=

### Cremi C unaligned, U-Net 3D

In [49]:
tf.reset_default_graph()

In [50]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/cremi/c/',
    dataset_constructor=dat.CREMIDataset,
    model_constructor=mod.UNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.UNET_3D_4LAYERS,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='unet-3d-4layers'
)

In [51]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='1')
classifier, dset_sampler = load_classifier(pipeline, run_name='1')

INFO:tensorflow:Restoring parameters from ./../../../datasets//cremi/c/results/unet_3d_4layers/run-1/model.ckpt
Model restored.


In [52]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('UVR-Net', df_cremi_c, p_error, rand_full, rand_merge, rand_split)

z: 0/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
z: 8/24
y: 0/1249
y: 150/1249
y: 300/1249
y: 450/1249
y: 600/1249
y: 750/1249
y: 900/1249
y: 1050/1249
y: 1089/1249
(24, 1249, 1249)
Begin segmentation
Segmentation complete
Performing foreground-restriction
Finding overlap matrix...
Completed in 7.773133 seconds
Calculating Rand F-Score...
Completed in 0.031238 seconds
UVR-Net
Pixel Error: 0.041922
Rand - Full: 0.748079
Rand - Merge: 0.933252
Rand - Split: 0.624223


In [53]:
with open('./../tex/tables/3d_snemi_results.tex', 'w') as latex_file:
    latex_file.write(df_snemi.to_latex(bold_rows=True))
with open('./../tex/tables/3d_cremi_a_results.tex', 'w') as latex_file:
    latex_file.write(df_cremi_a.to_latex(bold_rows=True))
with open('./../tex/tables/3d_cremi_b_results.tex', 'w') as latex_file:
    latex_file.write(df_cremi_b.to_latex(bold_rows=True))
with open('./../tex/tables/3d_cremi_c_results.tex', 'w') as latex_file:
    latex_file.write(df_cremi_c.to_latex(bold_rows=True))